# Feature engineering and exploration

In [1]:
import numpy as np
import pandas as pd
import datetime
import math
import gc

In [2]:
path='c:/users/datas/collab_filter/collab/'

In [3]:
train = pd.read_csv(path + 'train.csv')

In [49]:
??pd.read_csv

In [4]:
train.dtypes

msno                  object
song_id               object
source_system_tab     object
source_screen_name    object
source_type           object
target                 int64
dtype: object

In [5]:
test = pd.read_csv(path + 'test.csv')
test.dtypes

id                     int64
msno                  object
song_id               object
source_system_tab     object
source_screen_name    object
source_type           object
dtype: object

In [6]:
songs = pd.read_csv(path + 'songs.csv')
songs.dtypes

song_id         object
song_length      int64
genre_ids       object
artist_name     object
composer        object
lyricist        object
language       float64
dtype: object

In [8]:
songs.language=songs.language.astype('object')

In [9]:
songs.dtypes

song_id        object
song_length     int64
genre_ids      object
artist_name    object
composer       object
lyricist       object
language       object
dtype: object

In [10]:
members = pd.read_csv(path + 'members.csv', parse_dates=['registration_init_time','expiration_date'])
members.dtypes

msno                              object
city                               int64
bd                                 int64
gender                            object
registered_via                     int64
registration_init_time    datetime64[ns]
expiration_date           datetime64[ns]
dtype: object

In [12]:
songs_extra=pd.read_csv(path + 'song_extra_info.csv')
songs_extra.dtypes

song_id    object
name       object
isrc       object
dtype: object

In [13]:
train = train.merge(songs, on='song_id', how='left')
test = test.merge(songs, on='song_id', how='left')

In [14]:
members['membership_days'] = members['expiration_date'].subtract(members['registration_init_time']).dt.days.astype(int)

members['registration_year'] = members['registration_init_time'].dt.year
members['registration_month'] = members['registration_init_time'].dt.month
members['registration_date'] = members['registration_init_time'].dt.day

members['expiration_year'] = members['expiration_date'].dt.year
members['expiration_month'] = members['expiration_date'].dt.month
members['expiration_date'] = members['expiration_date'].dt.day
members = members.drop(['registration_init_time'], axis=1)

In [15]:
def isrc_to_year(isrc):
    if type(isrc) == str:
        if int(isrc[5:7]) > 17:
            return 1900 + int(isrc[5:7])
        else:
            return 2000 + int(isrc[5:7])
    else:
        return np.nan

In [16]:
songs_extra['song_year'] = songs_extra['isrc'].apply(isrc_to_year)
songs_extra.drop(['isrc', 'name'], axis = 1, inplace = True)

In [17]:
train = train.merge(members, on='msno', how='left')
test = test.merge(members, on='msno', how='left')

In [18]:
train = train.merge(songs_extra, on = 'song_id', how = 'left')
train.song_length.fillna(200000,inplace=True)
#train.song_length = train.song_length.astype(np.uint32)
#train.song_id = train.song_id.astype('category')


test = test.merge(songs_extra, on = 'song_id', how = 'left')
test.song_length.fillna(200000,inplace=True)
#test.song_length = test.song_length.astype(np.uint32)
#test.song_id = test.song_id.astype('category')

In [19]:
def genre_id_count(x):
    if x == 'no_genre_id':
        return 0
    else:
        return x.count('|') + 1

train['genre_ids'].fillna('no_genre_id',inplace=True)
test['genre_ids'].fillna('no_genre_id',inplace=True)
train['genre_ids_count'] = train['genre_ids'].apply(genre_id_count)
test['genre_ids_count'] = test['genre_ids'].apply(genre_id_count)
train.dtypes

msno                   object
song_id                object
source_system_tab      object
source_screen_name     object
source_type            object
target                  int64
song_length           float64
genre_ids              object
artist_name            object
composer               object
lyricist               object
language               object
city                    int64
bd                      int64
gender                 object
registered_via          int64
expiration_date         int64
membership_days         int32
registration_year       int64
registration_month      int64
registration_date       int64
expiration_year         int64
expiration_month        int64
song_year             float64
genre_ids_count         int64
dtype: object

In [23]:
train.lyricist.head(50)

0                                                   NaN
1                                                   NaN
2                                                   NaN
3                                                   NaN
4                                                   NaN
5                                                   NaN
6                                        TEDDY| TAEYANG
7                                                   NaN
8                                                   NaN
9                                                   NaN
10                                                  NaN
11                                                   嚴爵
12                                                  林怡鳳
13                                                  周杰倫
14                                          Lin Yi Feng
15                            Ji Hoon/Roco of Rocoberry
16                                                  吳汶芳
17                                              

In [24]:
def lyricist_count(x):
    if x == 'no_lyricist':
        return 0
    else:
        return sum(map(x.count, ['|', '/', '\\', ';'])) + 1

In [25]:
train['lyricist'].fillna('no_lyricist',inplace=True)
test['lyricist'].fillna('no_lyricist',inplace=True)
train['lyricists_count'] = train['lyricist'].apply(lyricist_count)
test['lyricists_count'] = test['lyricist'].apply(lyricist_count)

In [26]:
def composer_count(x):
    if x == 'no_composer':
        return 0
    else:
        return sum(map(x.count, ['|', '/', '\\', ';'])) + 1

train['composer'].fillna('no_composer',inplace=True)
test['composer'].fillna('no_composer',inplace=True)
train['composer_count'] = train['composer'].apply(composer_count)
test['composer_count'] = test['composer'].apply(composer_count)

In [27]:
def is_featured(x):
    if 'feat' in str(x) :
        return 1
    return 0

train['artist_name'].fillna('no_artist',inplace=True)
test['artist_name'].fillna('no_artist',inplace=True)
train['is_featured'] = train['artist_name'].apply(is_featured)
test['is_featured'] = test['artist_name'].apply(is_featured)

In [29]:
train.artist_name.tail(50)

7377368                             Twins
7377369                               薛之謙
7377370                               薛之謙
7377371                               薛之謙
7377372                     Trouble Maker
7377373                         G.E.M.鄧紫棋
7377374            迪克牛仔 (Dick and Cowboy)
7377375                   那對夫妻 (Nico&Kim)
7377376                             Heize
7377377                  蔡健雅 (Tanya Chua)
7377378                         BOX (BOX)
7377379                  蔡健雅 (Tanya Chua)
7377380                    詹森淮 (Riva Jan)
7377381                          Coldplay
7377382                          Coldplay
7377383                          Coldplay
7377384                    熱狗 (MC HotDog)
7377385                   Jung Joon Young
7377386                  鄭伊健 (Ekin Cheng)
7377387                        田馥甄 (Hebe)
7377388                   許茹芸 (Valen Hsu)
7377389                     孫盛希 (Shi Shi)
7377390                          Eric 周興哲
7377391              動力火車 (Power S

In [32]:
def artist_count(x):
    if x == 'no_artist':
        return 0
    else:
        return x.count('and') + x.count(',') + x.count('feat') + x.count('&')+1

train['artist_count'] = train['artist_name'].apply(artist_count)
test['artist_count'] = test['artist_name'].apply(artist_count)

In [33]:
train.artist_count.value_counts()

1     7171712
2      191852
3       12790
4         398
9         335
5         136
0         114
7          47
6          26
13          5
8           3
Name: artist_count, dtype: int64

In [34]:
train['artist_composer'] = (train['artist_name'] == train['composer']).astype(np.int8)
test['artist_composer'] = (test['artist_name'] == test['composer']).astype(np.int8)

In [35]:
train['artist_composer_lyricist'] = ((train['artist_name'] == train['composer']) & (train['artist_name'] == train['lyricist']) & (train['composer'] == train['lyricist'])).astype(np.int8)
test['artist_composer_lyricist'] = ((test['artist_name'] == test['composer']) & (test['artist_name'] == test['lyricist']) & (test['composer'] == test['lyricist'])).astype(np.int8)

In [36]:
train.language.dtype

dtype('O')

In [37]:
def song_lang_boolean(x):
    if '17.0' in str(x) or '45.0' in str(x):
        return 1
    return 0

train['song_lang_boolean'] = train['language'].apply(song_lang_boolean).astype(np.int8)
test['song_lang_boolean'] = test['language'].apply(song_lang_boolean).astype(np.int8)

In [38]:
_mean_song_length = np.mean(train['song_length'])
def smaller_song(x):
    if x < _mean_song_length:
        return 1
    return 0

train['smaller_song'] = train['song_length'].apply(smaller_song).astype(np.int8)
test['smaller_song'] = test['song_length'].apply(smaller_song).astype(np.int8)

In [39]:
_dict_count_song_played_train = {k: v for k, v in train['song_id'].value_counts().items()}
_dict_count_song_played_test = {k: v for k, v in test['song_id'].value_counts().items()}
def count_song_played(x):
    try:
        return _dict_count_song_played_train[x]
    except KeyError:
        try:
            return _dict_count_song_played_test[x]
        except KeyError:
            return 0

In [40]:
train['count_song_played'] = train['song_id'].apply(count_song_played).astype(np.int64)
test['count_song_played'] = test['song_id'].apply(count_song_played).astype(np.int64)

In [41]:
_dict_count_artist_played_train = {k: v for k, v in train['artist_name'].value_counts().items()}
_dict_count_artist_played_test = {k: v for k, v in test['artist_name'].value_counts().items()}
def count_artist_played(x):
    try:
        return _dict_count_artist_played_train[x]
    except KeyError:
        try:
            return _dict_count_artist_played_test[x]
        except KeyError:
            return 0

train['count_artist_played'] = train['artist_name'].apply(count_artist_played).astype(np.int64)
test['count_artist_played'] = test['artist_name'].apply(count_artist_played).astype(np.int64)

In [42]:
 train.columns

Index(['msno', 'song_id', 'source_system_tab', 'source_screen_name',
       'source_type', 'target', 'song_length', 'genre_ids', 'artist_name',
       'composer', 'lyricist', 'language', 'city', 'bd', 'gender',
       'registered_via', 'expiration_date', 'membership_days',
       'registration_year', 'registration_month', 'registration_date',
       'expiration_year', 'expiration_month', 'song_year', 'genre_ids_count',
       'lyricists_count', 'composer_count', 'is_featured', 'artist_count',
       'artist_composer', 'artist_composer_lyricist', 'song_lang_boolean',
       'smaller_song', 'count_song_played', 'count_artist_played'],
      dtype='object')

In [43]:
train.dtypes

msno                         object
song_id                      object
source_system_tab            object
source_screen_name           object
source_type                  object
target                        int64
song_length                 float64
genre_ids                    object
artist_name                  object
composer                     object
lyricist                     object
language                     object
city                          int64
bd                            int64
gender                       object
registered_via                int64
expiration_date               int64
membership_days               int32
registration_year             int64
registration_month            int64
registration_date             int64
expiration_year               int64
expiration_month              int64
song_year                   float64
genre_ids_count               int64
lyricists_count               int64
composer_count                int64
is_featured                 

In [45]:
train.head().T

,0,1,2,3,4
msno,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=
song_id,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=
source_system_tab,explore,my library,my library,my library,explore
source_screen_name,Explore,Local playlist more,Local playlist more,Local playlist more,Explore
source_type,online-playlist,local-playlist,local-playlist,local-playlist,online-playlist
target,1,1,1,1,1
song_length,206471,284584,225396,255512,187802
genre_ids,359,1259,1259,1019,1011
artist_name,Bastille,Various Artists,Nas,Soundway,Brett Young
composer,Dan Smith| Mark Crew,no_composer,N. Jones、W. Adams、J. Lordan、D. Ingle,Kwadwo Donkoh,Brett Young| Kelly Archer| Justin Ebach


In [46]:
train_new=train[['msno', 'song_id', 'target']]

In [48]:
test_new=test[['msno', 'song_id']]